In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import util
import time
import os 

from scipy.stats import pearsonr

### Define functions

In [2]:
#SEBASTIEN URCHS
def p_permut(empirical_value, permutation_values):
    n_permutation = len(permutation_values)
    if empirical_value >= 0:
        return (np.sum(permutation_values > empirical_value)+1) / (n_permutation + 1)
    return (np.sum(permutation_values < empirical_value)+1) / (n_permutation + 1)

### Define cases
Only loci with both DEL and DUP in dataset.

In [3]:
cases = ['1q21_1','2q13','13q12_12','15q11_2','16p11_2','16p13_11','17p12','22q11_2']

### Load null models and betamaps

In [4]:
n_path_nomc = '/home/harveyaa/Documents/fMRI/data/ukbb_9cohorts/null_models/{}_null_model_nomc.npy'

b_path_nomc = '/home/harveyaa/Documents/clara_paper/drop_maillard_15q11_2del/cc_{}_results_nomc.csv'

In [5]:
null_del = []
null_dup = []
beta_del = []
beta_dup = []
for c in cases:
    null_del.append(np.load(n_path_nomc.format('DEL'+c)))
    null_dup.append(np.load(n_path_nomc.format('DUP'+c)))
    
    beta_del.append(pd.read_csv(b_path_nomc.format('DEL'+c))['betas'])
    beta_dup.append(pd.read_csv(b_path_nomc.format('DUP'+c))['betas'])

### Get difference distributions
For each pair, difference between the mean of the real map of one and the null maps of the other and vice versa.

In [6]:
diff = np.zeros((len(cases),10000))
for i in range(len(cases)):
    for j in range(5000):
        beta = beta_del[i]
        null = null_dup[i][j]
        diff[i,j] = np.abs(np.mean(beta) - np.mean(null))

    for j in range(5000):
        beta = beta_dup[i]
        null = null_del[i][j]
        diff[i,j + 5000] = np.abs(np.mean(beta) - np.mean(null))

### Get actual differences
For each pair, difference between the mean of the betamaps.

In [8]:
diff_bb = np.zeros(len(cases))
for i in range(len(cases)):
    diff_bb[i] = np.abs(np.mean(beta_del[i]) - np.mean(beta_dup[i]))

### Calculate pvals

In [9]:
pval = []
for i in range(len(cases)):
    pval.append(p_permut(diff_bb[i],diff[i,:]))

In [10]:
pval

[0.23917608239176083,
 0.27787221277872215,
 0.3301669833016698,
 0.1857814218578142,
 0.14128587141285873,
 0.8995100489951005,
 0.1934806519348065,
 0.0313968603139686]